In [1]:
#importations
import requests
import random
from random import randrange
import json
#import nltk
from langdetect import detect
import os
import glob
import gensim.downloader as api
from nltk.corpus import stopwords

In [2]:
def write_json(chemin,contenu):
    with open(chemin,"w",encoding="utf-8") as w :
        w.write(json.dumps(contenu, indent=2, ensure_ascii=False))
def ouvrir_json(chemin) :
    with open(chemin,encoding="utf-8") as f :
        toto = json.load(f)
    return toto

In [4]:
def requete_base(lang) : 
    nom = "requetes_chansons_"+lang+".json"
    if os.path.exists(nom) :
        return nom
    langues = {"fr":"Q150","en" :"Q1860" }
    #récupérer aussi l'auteur ?
    requete_all = """
    SELECT DISTINCT ?chanson ?chansonLabel ?performer ?performerLabel WHERE
    {
      ?chanson wdt:P7937 wd:Q7366;
               wdt:P175 ?performer;
               rdfs:label ?chansonLabel;
               wdt:P407 wd:%s.
       ?performer rdfs:label ?performerLabel.
      FILTER(lang(?chansonLabel)="%s").
      FILTER(lang(?performerLabel)="%s").
  }
    """%(langues[lang],lang,lang)

    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': requete_all, 'format': 'json'}).json()
    
    write_json(nom,data)
    return nom

In [6]:
def structuration_requete(fichier) :

    resultats_json = ouvrir_json(fichier)
    liste_niv1 = resultats_json["results"]["bindings"]
    
    resultats = {}
    chanteur = {}
   
    for un in liste_niv1 :
        if un['chanson']['value'] not in resultats :
            k = un['chanson']['value'].split("http://www.wikidata.org/entity/")[1]
            resultats[k] = (un['chansonLabel']['value'],un['performerLabel']['value'])
    """        
    for un in liste_niv1 :
        if un['performerLabel']['value'] not in chanteur :
            chanteur[un['performerLabel']['value']] = [un['chansonLabel']['value']]
        else :
            chanteur[un['performerLabel']['value']].append(un['chansonLabel']['value'])
   """     
    return resultats

In [8]:
def choix_reponse(dico,model) :
    estBon = False
    while estBon == False :
        estBon = True
        #je choisis une chanson aléatoirement
        cle = random.choice(list(dico.keys()))
        rep_poss = dico[cle][0]
        
        #minuscule pour homogénéiser avec la liste de stopwords qui est miniscule
        reponse = rep_poss.lower()
    
        
        #je tokenize ma chanson
        tokens = reponse.split()
        
        #je vérifie que chaque token est bien dans le modèle (exception pour gérer le KeyError)
        try :
            for t in tokens :
                model.most_similar(positive=[t])
        except KeyError :
            estBon = False
            pass
    
    question = ""
    tokens = reponse.split()
    
    for t in tokens :
        if t not in stopwords.words('english'):
            question+=model.most_similar(positive=[t])[0][0]+" "
        else :
            question+=t+" "
    return question,reponse,dico[cle][1]
    

In [10]:
def jeu() :
    
    print("Chargement...")
    model = api.load("word2vec-google-news-300") 
    fichier = requete_base("en")
    resultats =  structuration_requete(fichier)
    
    cpt = 0
    while cpt < 5 :
        question,reponse,chanteur = choix_reponse(resultats,model)
        print("Chanteur : ",chanteur)
        print("Jeu de mots : ",question)
        prop = input("Quelle est votre proposition ?")
        if prop.lower() != reponse :
            print("Non ! La bonne réponse est ",reponse)
            print("-"*100)
        else :
            print("Bonne réponse !")
            print("-"*100)
        cpt+=1
    return

In [11]:
jeu()

Chargement...
Chanteur :  April Wine
Jeu de mots :  sobbing wife 
Quelle est votre proposition ?crying woman
Non ! La bonne réponse est  weeping widow
----------------------------------------------------------------------------------------------------
Chanteur :  Metallica
Jeu de mots :  the three thoroughbred_horsemen 
Quelle est votre proposition ?the four horsemen
Bonne réponse !
----------------------------------------------------------------------------------------------------
Chanteur :  Judas Priest
Jeu de mots :  saint in heck 
Quelle est votre proposition ?
Non ! La bonne réponse est  saints in hell
----------------------------------------------------------------------------------------------------
Chanteur :  Sheryl Crow
Jeu de mots :  soaking up the sunlight 
Quelle est votre proposition ?
Non ! La bonne réponse est  soak up the sun
----------------------------------------------------------------------------------------------------
Chanteur :  Creed Fisher
Jeu de mots :  kis